<a href="https://colab.research.google.com/github/EMcoding17/ExtraccionConocimientosDeBD/blob/main/Actividad04/CarPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANÁLISIS DEL DATASET CAR PRICE**
## **Extraccion de conocimientos de base de datos**
## **Romero Sosa Emma Yuridia**
## **IDGS 91** 


---



In [177]:
!pip install fuzzywuzzy

# **Librerías**

In [178]:
import numpy as np
import pandas as pd
#librerias para crear graficos
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
#Librerias para valores que coinciden
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# **Cargar dataset**


In [179]:
#Cargar dataset desde github
!wget https://raw.githubusercontent.com/EMcoding17/ExtraccionConocimientosDeBD/main/Actividad04/car_price_assignment.csv
dfcar_price = pd.read_csv("car_price_assignment.csv",delimiter=',',header='infer')

#Mostrar los primeros 5 datos
dfcar_price.head()

--2021-10-24 00:06:24--  https://raw.githubusercontent.com/EMcoding17/ExtraccionConocimientosDeBD/main/Actividad04/car_price_assignment.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26717 (26K) [text/plain]
Saving to: ‘car_price_assignment.csv.10’

car_price_assignmen 100%[===================>]  26.09K  --.-KB/s    in 0.001s  

2021-10-24 00:06:24 (19.0 MB/s) - ‘car_price_assignment.csv.10’ saved [26717/26717]



,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,carwidth,carheight,curbweight,enginetype,cylindernumber,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [180]:
# Obtener informacion del DataFrame
dfcar_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

# **Funciones**

## **Funciones para la limpieza de datos**

In [181]:
# Función para conocer los valores unicos de las columnas.
def uniqueData(df, col):
  # Obtener todos los valores unicos en la columna
  lista_unica = df[col].unique()

  # sort them alphabetically and then take a closer look
  lista_unica.sort()
  print("Columna: ", col)
  print("No. columnas unicas: ", lista_unica.size)
  print("Valores unicos: ", lista_unica, "\n ")


In [182]:
#funcion para convertir valores a minusculas y eliminar espacios blancos al inicio y final de los valores
def correctionData(df, col):
  # Convertir los valores a minusculas de la columna CarName
  dfcar_price['CarName'] = dfcar_price['CarName'].str.lower()

  # Eliminar los espacios en blanco al inicio y final de los valores
  dfcar_price['CarName'] = dfcar_price['CarName'].str.strip()
  print("Corrección de datos realizada! \n")

In [183]:
# función para mostrar los valores que coincidan con un string y reemplazarlos
def matchData(df, col, string_to_match):
    # Obtener lista de valores unicos
    valores_unicos = df[col].unique()
    
    # Obtener las 5 coincidencias mas cercanas al string
    matches = fuzzywuzzy.process.extract(string_to_match, valores_unicos, 
                                         limit=5, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Obtener solo los valores con coincidencia mayor a 90
    close_matches = [matches[0] for matches in matches if matches[1] >= 75]

    # Obtener las filas de los valores que coinciden
    rows_with_matches = df[col].isin(close_matches)

    # Reemplazar todas las filas de los valores que coinciden
    df.loc[rows_with_matches, col] = string_to_match
    
    #Mostrar coincidencias
    print("Coincidencias: ", matches)
    print("Coincidencias mayoreas a 75%: ", close_matches)
    print("Filas modificadas: ", rows_with_matches.sum())
    print("\n ")

# **Limpieza de datos**


## **Verificar datos faltantes**

In [184]:
#Obtener la suma de los valores perdidos por cada columna
datos_perdidos = dfcar_price.isnull().sum()
datos_perdidos

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

## **Verificar datos duplicados**

In [185]:
#Verificar que no haya datos duplicados dentro del dataframe
dfcar_price.duplicated().sum()

0

## **Corrección de entrada de datos inconsistente**

In [186]:
colm = ["CarName", "fueltype", "aspiration", "doornumber", "carbody", "drivewheel", "enginelocation", "enginetype", "cylindernumber", "fuelsystem"]

for i in range(len(colm)):
  col = colm[i]
  uniqueData(dfcar_price, col)

Columna:  CarName
No. columnas unicas:  147
Valores unicos:  ['Nissan versa' 'alfa-romero Quadrifoglio' 'alfa-romero giulia'
 'alfa-romero stelvio' 'audi 100 ls' 'audi 100ls' 'audi 4000' 'audi 5000'
 'audi 5000s (diesel)' 'audi fox' 'bmw 320i' 'bmw x1' 'bmw x3' 'bmw x4'
 'bmw x5' 'bmw z4' 'buick century' 'buick century luxus (sw)'
 'buick century special' 'buick electra 225 custom'
 'buick opel isuzu deluxe' 'buick regal sport coupe (turbo)'
 'buick skyhawk' 'buick skylark' 'chevrolet impala'
 'chevrolet monte carlo' 'chevrolet vega 2300' 'dodge challenger se'
 'dodge colt (sw)' 'dodge colt hardtop' 'dodge coronet custom'
 'dodge coronet custom (sw)' 'dodge d200' 'dodge dart custom'
 'dodge monaco (sw)' 'dodge rampage' 'honda accord' 'honda accord cvcc'
 'honda accord lx' 'honda civic' 'honda civic (auto)' 'honda civic 1300'
 'honda civic 1500 gl' 'honda civic cvcc' 'honda prelude' 'isuzu D-Max '
 'isuzu D-Max V-Cross' 'isuzu MU-X' 'jaguar xf' 'jaguar xj' 'jaguar xk'
 'maxda glc deluxe

> ### **Columna: CarName**

Podemos observar que existen algunos datos inconsistentes en la columna **CarName** como **'audi 100 ls'** y **'audi 100ls'** donde el primero cuenta con espacio entre 100 y ls, mientras que el segundo no. Otro ejemplo es con **'Nissan versa'** que comienza con mayusculas, mientras que la mayor cantidad de nombres son en minisculas 

Asi que corregimos estos datos de entrada de la siguiente forma:
* Convertimos todos los valores a minusculas.
* Eliminamos espacios en blanco al inicio y al final de los valores.
* Reemplazamos los valores que creemos que coinciden.

In [187]:
# Corregir datos
correctionData(dfcar_price, "CarName")

#Reemplazar datos que coinciden en la columna CarName
matchData(dfcar_price, 'CarName', "audi 100 ls")

#Verificar modificaciones
uniqueData(dfcar_price, 'CarName')

Corrección de datos realizada! 

Coincidencias:  [('audi 100 ls', 100), ('audi 100ls', 76), ('audi 5000', 70), ('audi 4000', 70), ('audi 5000s (diesel)', 64)]
Coincidencias mayoreas a 75%:  ['audi 100 ls', 'audi 100ls']
Filas modificadas:  3

 
Columna:  CarName
No. columnas unicas:  146
Valores unicos:  ['alfa-romero giulia' 'alfa-romero quadrifoglio' 'alfa-romero stelvio'
 'audi 100 ls' 'audi 4000' 'audi 5000' 'audi 5000s (diesel)' 'audi fox'
 'bmw 320i' 'bmw x1' 'bmw x3' 'bmw x4' 'bmw x5' 'bmw z4' 'buick century'
 'buick century luxus (sw)' 'buick century special'
 'buick electra 225 custom' 'buick opel isuzu deluxe'
 'buick regal sport coupe (turbo)' 'buick skyhawk' 'buick skylark'
 'chevrolet impala' 'chevrolet monte carlo' 'chevrolet vega 2300'
 'dodge challenger se' 'dodge colt (sw)' 'dodge colt hardtop'
 'dodge coronet custom' 'dodge coronet custom (sw)' 'dodge d200'
 'dodge dart custom' 'dodge monaco (sw)' 'dodge rampage' 'honda accord'
 'honda accord cvcc' 'honda accord lx' '

## **Verificar tipo de datos**

In [188]:
#Verificacion del tipo de dato por columna
dfcar_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1